In [1]:
import pandas as pd
from datetime import datetime
import mca
import xgboost as xgb
from xgboost import XGBRegressor, plot_importance
from sklearn.preprocessing import LabelEncoder

C:\Users\mingt\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#Split the file since its too big for github, extract the file outside of .git repository

hpg_res = pd.read_csv('C:\\Users\\mingt\\Downloads\\hpg_reserve.csv\\hpg_reserve.csv')
hpg_res.info()
df1 = hpg_res.iloc[0:100160]
df2 = hpg_res.iloc[100161:200320]
#df1.to_csv('hpg_reserve1.csv')
#df2.to_csv('hpg_reserve2.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000320 entries, 0 to 2000319
Data columns (total 4 columns):
hpg_store_id        object
visit_datetime      object
reserve_datetime    object
reserve_visitors    int64
dtypes: int64(1), object(3)
memory usage: 61.0+ MB


In [2]:
air_reserve = pd.read_csv('air_reserve.csv')
air_store_info= pd.read_csv('air_store_info.csv')
air_visit_data = pd.read_csv('air_visit_data.csv')
date_info = pd.read_csv('date_info.csv')
hpg_store_info = pd.read_csv('hpg_store_info.csv')
store_id_relation = pd.read_csv('store_id_relation.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [3]:
#concat files back after its been uploaded on github
hpg_reserve1 = pd.read_csv('hpg_reserve1.csv')
hpg_reserve2 = pd.read_csv('hpg_reserve2.csv')

hpg_reserve = pd.concat([hpg_reserve1, hpg_reserve2])
hpg_reserve['visit_datetime'] = pd.to_datetime(hpg_reserve['visit_datetime'])
hpg_reserve['reserve_datetime'] = pd.to_datetime(hpg_reserve['reserve_datetime'])

In [4]:
air_combined = pd.merge(air_reserve, air_store_info, on='air_store_id', how='outer')
hpg_combined = pd.merge(hpg_reserve, hpg_store_info, on='hpg_store_id', how='left')

air_combined.head(5)
#hpg_combined.head(5)

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,air_genre_name,air_area_name,latitude,longitude
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
1,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
2,air_877f79706adbfb06,2016-01-02 18:00:00,2016-01-01 16:00:00,2.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
3,air_877f79706adbfb06,2016-01-02 21:00:00,2016-01-01 16:00:00,2.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
4,air_877f79706adbfb06,2016-01-08 21:00:00,2016-01-04 20:00:00,2.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


In [5]:
df = store_id_relation.merge(hpg_combined, on='hpg_store_id', how='left')

df2 = air_combined.merge(df, on='air_store_id', how='left')
lbl = LabelEncoder()

df2['latitude'] = df2['latitude_x'].astype(str)
df2['longitude'] = df2['longitude_x'].astype(str)

date_info['calendar_date'] = pd.to_datetime(date_info['calendar_date']).dt.date.astype(str)
#date_info['holiday_flg'] = date_info['holiday_flg'].map({1: 'Yes', 0: 'No'})
date_info['days_of_week'] = lbl.fit_transform(date_info['day_of_week'])
date_info['weekend_or_weekday'] = date_info['day_of_week'].map({'Monday': 0, 
                                                                'Tuesday': 0, 
                                                                'Wednesday': 0, 
                                                                'Thursday': 0, 
                                                                'Friday': 0,
                                                                'Saturday': 1, 
                                                                'Sunday': 1})


#date_info['weekend_or_weekday'] = lbl.fit_transform(date_info['weekend_or_weekday'])

In [6]:
sub_store = sample_submission['id'].apply(lambda x: str(x).split('_', 2)[:2])
sub_dates = pd.to_datetime(sample_submission['id'].apply(lambda x: str(x).split('_', 2)[2]).rename('Date'))
sub_stores = pd.Series(['_'.join(x) for x in sub_store]).rename('air_store_id')

sub_dt = pd.DataFrame({
        'restaurant': sub_stores,
        'date': sub_dates.dt.date.astype(str),
        'year': sub_dates.dt.year,
        'month': sub_dates.dt.month,
        'day': sub_dates.dt.day })

sub_df = pd.concat([sample_submission, sub_dt], axis=1)
#sub_df['weekend_or_weekday'] = sub_df['weekend_or_weekday'].apply(LabelEncoder().fit_transform)
sub_df2 = sub_df.merge(date_info, left_on= sub_df['date'], right_on=date_info['calendar_date'])
sub_df2.drop(['day_of_week', 'year', 'calendar_date', 'date'], axis =1)

sub_target = sample_submission['visitors']

In [26]:
a = pd.to_datetime(df2['visit_datetime_x'])
b = pd.to_datetime(df2['reserve_datetime_x'])
c = pd.to_datetime(df2['visit_datetime_y'])
d = pd.to_datetime(df2['reserve_datetime_y'])

datetime_df =pd.DataFrame({
        'air_visit_date': a.dt.date.astype(str),
        'year': a.dt.year,
        'month': a.dt.month,
        'day': a.dt.day,
        'air_visit_hour': a.dt.hour,
        'air_reserve_date': b.dt.date,
        'air_reserve_year': b.dt.year,
        'air_reserve_month': b.dt.month,
        'air_reserve_day': b.dt.day,
        'air_reserve_hour': b.dt.hour,
         
        'hpg_visit_date': c.dt.date,
        'hpg_visit_year': c.dt.year,
        'hpg_visit_month': c.dt.month,
        'hpg_visit_day': c.dt.day,
        'hpg_visit_hour': c.dt.hour,
        'hpg_reserve_date': d.dt.date,
        'hpg_reserve_year': d.dt.year,
        'hpg_reserve_month': d.dt.month,
        'hpg_reserve_day': d.dt.day,
        'hpg_reserve_hour': d.dt.hour        
    }).fillna(0)

datetime_df2 = datetime_df.merge(date_info, left_on=datetime_df['air_visit_date'], 
                                 right_on=date_info['calendar_date'])


categorical_df = df2[['air_store_id', 'air_genre_name','air_area_name','latitude','longitude',
                      'hpg_genre_name','hpg_area_name']].fillna('None_Stated').apply(LabelEncoder().fit_transform)

#categorical_df2 = pd.get_dummies(df2[['air_genre_name','air_area_name','latitude','longitude',
                      #'hpg_genre_name','hpg_area_name']].fillna('None_Stated'))

air_visitors = df2['reserve_visitors_x'].fillna(0)
hpg_visitors = df2['reserve_visitors_y'].fillna(0)
df2['reserve_total'] = air_visitors + hpg_visitors

combined_visitors = df2['reserve_total'].astype(int).rename('visitors')

with_visitors = pd.concat([combined_visitors, datetime_df2[['month', 'day', 'days_of_week', 'holiday_flg', 'weekend_or_weekday']], categorical_df], axis=1)
features = pd.concat([df2['air_store_id'], datetime_df2[['month', 'day', 'days_of_week', 'holiday_flg', 'weekend_or_weekday']], categorical_df], axis=1)

Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x000001B9DE237860>>
Traceback (most recent call last):
  File "C:\Users\mingt\AppData\Local\Continuum\Anaconda3\lib\site-packages\xgboost\core.py", line 324, in __del__
    _check_call(_LIB.XGDMatrixFree(self.handle))
AttributeError: 'DMatrix' object has no attribute 'handle'


In [27]:
with_visitors

,visitors,month,day,days_of_week,holiday_flg,weekend_or_weekday,air_store_id,air_genre_name,air_area_name,latitude,longitude,hpg_genre_name,hpg_area_name
0,1,1.0,1.0,0.0,1.0,0.0,426,8,62,54,83,10,16
1,2,1.0,1.0,0.0,1.0,0.0,426,8,62,54,83,10,16
2,2,1.0,1.0,0.0,1.0,0.0,426,8,62,54,83,10,16
3,2,1.0,1.0,0.0,1.0,0.0,426,8,62,54,83,10,16
4,2,1.0,1.0,0.0,1.0,0.0,426,8,62,54,83,10,16
5,2,1.0,1.0,0.0,1.0,0.0,426,8,62,54,83,10,16
6,3,1.0,1.0,0.0,1.0,0.0,426,8,62,54,83,10,16
7,2,1.0,1.0,0.0,1.0,0.0,426,8,62,54,83,10,16
8,4,1.0,1.0,0.0,1.0,0.0,426,8,62,54,83,10,16
9,2,1.0,1.0,0.0,1.0,0.0,426,8,62,54,83,10,16


In [16]:
features.head(15)

,air_store_id,month,day,days_of_week,holiday_flg,weekend_or_weekday,air_genre_name,air_area_name,latitude,longitude,hpg_genre_name,hpg_area_name
0,air_877f79706adbfb06,1.0,1.0,0.0,1.0,0.0,8,62,54,83,10,16
1,air_877f79706adbfb06,1.0,1.0,0.0,1.0,0.0,8,62,54,83,10,16
2,air_877f79706adbfb06,1.0,1.0,0.0,1.0,0.0,8,62,54,83,10,16
3,air_877f79706adbfb06,1.0,1.0,0.0,1.0,0.0,8,62,54,83,10,16
4,air_877f79706adbfb06,1.0,1.0,0.0,1.0,0.0,8,62,54,83,10,16
5,air_877f79706adbfb06,1.0,1.0,0.0,1.0,0.0,8,62,54,83,10,16
6,air_877f79706adbfb06,1.0,1.0,0.0,1.0,0.0,8,62,54,83,10,16
7,air_877f79706adbfb06,1.0,1.0,0.0,1.0,0.0,8,62,54,83,10,16
8,air_877f79706adbfb06,1.0,1.0,0.0,1.0,0.0,8,62,54,83,10,16
9,air_877f79706adbfb06,1.0,1.0,0.0,1.0,0.0,8,62,54,83,10,16


In [15]:
len(features['air_store_id'].value_counts())


829

In [28]:
by_mean = with_visitors.groupby(['air_store_id', 'month', 'day', 'days_of_week', 'air_genre_name','air_area_name','latitude','longitude',
                      'hpg_genre_name','hpg_area_name'], as_index=False)['visitors'].mean()
by_max = with_visitors.groupby(['air_store_id', 'month', 'day', 'days_of_week', 'air_genre_name','air_area_name','latitude','longitude',
                      'hpg_genre_name','hpg_area_name'], as_index=False)['visitors'].max()
by_median = with_visitors.groupby(['air_store_id', 'month', 'day', 'days_of_week', 'air_genre_name','air_area_name','latitude','longitude',
                      'hpg_genre_name','hpg_area_name'], as_index=False)['visitors'].median()
by_all = with_visitors.groupby(['air_store_id', 'month', 'day', 'days_of_week', 'air_genre_name','air_area_name','latitude','longitude',
                      'hpg_genre_name','hpg_area_name'], as_index=False)['visitors'].mean()


In [16]:
final_df = by_month.merge(by_day,on='air_store_id', how='left').merge(by_dow,on='air_store_id', how='left').fillna(0)

In [29]:
by_all

,air_store_id,month,day,days_of_week,air_genre_name,air_area_name,latitude,longitude,hpg_genre_name,hpg_area_name,visitors
0,0,2.0,12.0,3.0,6,44,68,85,10,16,8.690476
1,1,5.0,28.0,2.0,6,62,54,83,10,16,4.181818
2,1,6.0,2.0,4.0,6,62,54,83,10,16,3.316901
3,4,7.0,17.0,3.0,2,102,26,26,10,16,6.642857
4,7,12.0,10.0,2.0,7,15,15,15,10,16,6.554094
5,7,12.0,12.0,1.0,7,15,15,15,10,16,6.375643
6,8,5.0,27.0,0.0,7,66,89,53,10,16,17.333333
7,10,1.0,6.0,0.0,7,22,104,100,8,15,7.160078
8,10,1.0,7.0,2.0,7,22,104,100,8,15,7.930218
9,10,1.0,8.0,3.0,7,22,104,100,8,15,8.590909


In [59]:

id_str = pd.Series([str(x) for x in final_df['air_store_id']]).rename('id')
    

In [62]:
for i in id_str[:10]:
    print(type(i))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [30]:
#X_train, X_test, y_train, y_test = train_test_split(features, target, 
                                                    #test_size=0.1, random_state=7)

dmatrix = xgb.DMatrix(by_all, by_all['visitors'])

xgb_params_cv = dict(eta=0.1,
                  objective='reg:linear',
                  eval_metric='rmse')
best = xgb.cv(params = xgb_params_cv, dtrain=dmatrix, num_boost_round=1000, early_stopping_rounds=100, verbose_eval=True)

best.tail(5)               

[0]	train-rmse:7.48717+0.0691975	test-rmse:7.48888+0.160042
[1]	train-rmse:6.77561+0.0647875	test-rmse:6.77859+0.165865
[2]	train-rmse:6.13452+0.0610063	test-rmse:6.13679+0.172989
[3]	train-rmse:5.55463+0.0574608	test-rmse:5.56212+0.17124
[4]	train-rmse:5.03154+0.0540901	test-rmse:5.03868+0.170752
[5]	train-rmse:4.55869+0.0504776	test-rmse:4.56984+0.175687
[6]	train-rmse:4.13134+0.0474383	test-rmse:4.14558+0.176656
[7]	train-rmse:3.74563+0.0456675	test-rmse:3.76246+0.176519
[8]	train-rmse:3.39707+0.0428561	test-rmse:3.41315+0.181296
[9]	train-rmse:3.08122+0.0405648	test-rmse:3.09467+0.185952
[10]	train-rmse:2.7956+0.038396	test-rmse:2.81019+0.191608
[11]	train-rmse:2.53741+0.0356643	test-rmse:2.55532+0.193101
[12]	train-rmse:2.30379+0.0333644	test-rmse:2.32514+0.197081
[13]	train-rmse:2.0925+0.0311328	test-rmse:2.11721+0.200847
[14]	train-rmse:1.90155+0.0288419	test-rmse:1.92955+0.204798
[15]	train-rmse:1.72878+0.0266871	test-rmse:1.76141+0.208537
[16]	train-rmse:1.57248+0.0244376	test

,test-rmse-mean,test-rmse-std,train-rmse-mean,train-rmse-std
55,0.266809,0.121769,0.064290,0.001985
56,0.266361,0.119204,0.060149,0.001878
57,0.266048,0.116761,0.056366,0.001735
58,0.266412,0.114378,0.052823,0.001651
59,0.266018,0.112422,0.049610,0.001513
